## Análise de produtos e categorias vendidas juntas
 - Usaremos o algoritmo Apriori
 - Basicamente encontraremos quais produtos tendem a ser vendidos no mesmo pedido
 - Depois olharemos quais categorias são vendidas nos mesmos pedidos
 - Os dados utilizados são do Olist e estão disponíveis no link: https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce

#### Importando bibliotecas
Usaremos pandas e as bibliotecas necessárias ao algoritmo Apriori

In [2]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

#### Vamos carregar as bases de dados
Vamos usar a planilha de itens/pedidos e a de produtos<br>
A base de produtos possui a categoria de cada item

In [5]:
itens_pedidos = pd.read_csv(r'C:\Users\elise\OneDrive\Área de Trabalho\olist\olist_order_items_dataset.csv')
produtos = pd.read_csv(r'C:\Users\elise\OneDrive\Área de Trabalho\olist\olist_products_dataset.csv')

In [146]:
#Vamos dar uma olhada nas bases de dados:
display(itens_pedidos.head())

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,contagem
0,0008288aa423d2a3f00fcb17cd7d8719,1,368c6c730842d78016ad823897a372db,1f50f920176fa81dab994f9023523100,2018-02-21 02:55:52,49.90,13.37,2
1,0008288aa423d2a3f00fcb17cd7d8719,2,368c6c730842d78016ad823897a372db,1f50f920176fa81dab994f9023523100,2018-02-21 02:55:52,49.90,13.37,2
2,00143d0f86d6fbd9f9b38ab440ac16f5,1,e95ee6822b66ac6058e2e4aff656071a,a17f621c590ea0fab3d5d883e1630ec6,2017-10-20 16:07:52,21.33,15.10,3
3,00143d0f86d6fbd9f9b38ab440ac16f5,2,e95ee6822b66ac6058e2e4aff656071a,a17f621c590ea0fab3d5d883e1630ec6,2017-10-20 16:07:52,21.33,15.10,3
4,00143d0f86d6fbd9f9b38ab440ac16f5,3,e95ee6822b66ac6058e2e4aff656071a,a17f621c590ea0fab3d5d883e1630ec6,2017-10-20 16:07:52,21.33,15.10,3


In [145]:
display(produtos.head())

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0


#### Removendo pedidos com apenas um item
Como iremos olhar a associação conjunta de produtos, usaremos apenas os pedidos que possuem mais de um item incluso

In [147]:
contagem = itens_pedidos.order_id.value_counts()
itens_pedidos['contagem'] = itens_pedidos['order_id'].map(contagem)
itens_pedidos = itens_pedidos.query('contagem > 1').reset_index(drop = True)

#### Agora que nossa base possui apenas pedidos com mais de um item, vamos trazer a categoria da base de produtos

In [36]:
# Vamos reduzir a dimensionalidade da base e incluir o nome da categoria
df = itens_pedidos.merge(right = produtos, on = 'product_id')[['order_id', 'product_id', 'product_category_name']]
df.order_id = df.order_id.astype(str)
df.product_id = df.product_id.astype(str)
df.product_category_name = df.product_category_name.astype(str)
df.head()

,order_id,product_id,product_category_name
0,0008288aa423d2a3f00fcb17cd7d8719,368c6c730842d78016ad823897a372db,ferramentas_jardim
1,0008288aa423d2a3f00fcb17cd7d8719,368c6c730842d78016ad823897a372db,ferramentas_jardim
2,042e14b2dcf81a8468a49c14d28036c9,368c6c730842d78016ad823897a372db,ferramentas_jardim
3,042e14b2dcf81a8468a49c14d28036c9,368c6c730842d78016ad823897a372db,ferramentas_jardim
4,095234196e1f3c4feadc4dca677b3971,368c6c730842d78016ad823897a372db,ferramentas_jardim


In [150]:
#Vamos ver quais são as categorias mais comuns na nossa base de dados
pd.DataFrame(df.product_category_name.value_counts()).head(15)

,product_category_name
moveis_decoracao,3272
cama_mesa_banho,3152
informatica_acessorios,1933
utilidades_domesticas,1851
esporte_lazer,1696
beleza_saude,1518
ferramentas_jardim,1430
relogios_presentes,671
moveis_escritorio,659
telefonia,636


#### Agora vamos usar o algoritmo Apriori para encontrar categorias com maior relação entre si

In [83]:
# Precisamos criar uma matriz de transações
transacoes = df.groupby('order_id')['product_category_name'].apply(list).values.tolist()

# Transformar as transações em uma matriz binária
te = TransactionEncoder()
te_ary = te.fit(transacoes).transform(transacoes)
df_trans = pd.DataFrame(te_ary, columns=te.columns_)

# Encontrar itens frequentes
itens_frequentes = apriori(df_trans, min_support=0.0007, use_colnames=True)

# Encontrar as regras de associação
regras_categoria = association_rules(itens_frequentes, metric='lift', min_threshold=1)

In [104]:
#Vamos plotar o resultado ordenando pela Confiança, que define a "força" da associação
regras_categoria.sort_values(by = 'confidence', ascending = False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
6,(casa_conforto),(cama_mesa_banho),0.007651,0.148322,0.004386,0.573333,3.865465,0.003252,1.996120
11,(construcao_ferramentas_iluminacao),(moveis_decoracao),0.004998,0.141487,0.001122,0.224490,1.586643,0.000415,1.107030
2,(cool_stuff),(bebes),0.018464,0.021932,0.002040,0.110497,5.038160,0.001635,1.099567
3,(bebes),(cool_stuff),0.021932,0.018464,0.002040,0.093023,5.038160,0.001635,1.082207
1,(bebes),(brinquedos),0.021932,0.023972,0.001938,0.088372,3.686432,0.001412,1.070643
15,(nan),(utilidades_domesticas),0.016730,0.078649,0.001428,0.085366,1.085397,0.000112,1.007343
0,(brinquedos),(bebes),0.023972,0.021932,0.001938,0.080851,3.686432,0.001412,1.064102
9,(casa_construcao),(ferramentas_jardim),0.009793,0.061308,0.000714,0.072917,1.189355,0.000114,1.012522
12,(fashion_bolsas_e_acessorios),(nan),0.015301,0.016730,0.000918,0.060000,3.586463,0.000662,1.046032
13,(nan),(fashion_bolsas_e_acessorios),0.016730,0.015301,0.000918,0.054878,3.586463,0.000662,1.041875


#### Agora vamos analisar as associações a nível SKU
Para isso, podemos usar o mesmo script, mas mudando o parâmetro da matriz de transações

In [139]:
# Precisamos criar uma matriz de transações
transacoes = df.groupby('order_id')['product_id'].apply(list).values.tolist()

# Transformar as transações em uma matriz binária
te = TransactionEncoder()
te_ary = te.fit(transacoes).transform(transacoes)
df_trans = pd.DataFrame(te_ary, columns=te.columns_)

# Encontrar itens frequentes
itens_frequentes = apriori(df_trans, min_support=0.0008, use_colnames=True)

# Encontrar as regras de associação
regras_sku = association_rules(itens_frequentes, metric='lift', min_threshold=1)

In [140]:
#Vamos plotar o resultado ordenando pela Confiança, que define a "força" da associação
regras_sku.sort_values(by = 'confidence', ascending = False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
12,(f4f67ccaece962d013a4e1d7dc3a61f7),(4fcb3d9a5f4871e8362dfedbdb02b064),0.001836,0.001938,0.001734,0.944444,487.283626,0.001731,17.965113
13,(4fcb3d9a5f4871e8362dfedbdb02b064),(f4f67ccaece962d013a4e1d7dc3a61f7),0.001938,0.001836,0.001734,0.894737,487.283626,0.001731,9.482556
1,(35afc973633aaeb6b877ff57b2793310),(99a4788cb24856965c36a24e339b6058),0.004182,0.007141,0.002958,0.707317,99.054704,0.002928,3.392269
6,(36f60d45225e60c7da4558b070ce4b60),(e53e557d5a159f5aa2c5e995dfdf244b),0.005202,0.005713,0.003468,0.666667,116.702381,0.003439,2.982862
7,(e53e557d5a159f5aa2c5e995dfdf244b),(36f60d45225e60c7da4558b070ce4b60),0.005713,0.005202,0.003468,0.607143,116.702381,0.003439,2.532212
4,(3f14d740544f37ece8a9e7bc8349797e),(36f60d45225e60c7da4558b070ce4b60),0.002244,0.005202,0.001224,0.545455,104.844920,0.001212,2.188555
0,(99a4788cb24856965c36a24e339b6058),(35afc973633aaeb6b877ff57b2793310),0.007141,0.004182,0.002958,0.414286,99.054704,0.002928,1.700176
5,(36f60d45225e60c7da4558b070ce4b60),(3f14d740544f37ece8a9e7bc8349797e),0.005202,0.002244,0.001224,0.235294,104.844920,0.001212,1.304758
8,(389d119b48cf3043d311335e499d9c6b),(422879e10f46682990de24d770e7f83d),0.008773,0.010507,0.001122,0.127907,12.173515,0.001030,1.134619
9,(422879e10f46682990de24d770e7f83d),(389d119b48cf3043d311335e499d9c6b),0.010507,0.008773,0.001122,0.106796,12.173515,0.001030,1.109743
